In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd

In [2]:
df=pd.read_csv("Cellphone.csv")

In [3]:
df=df.drop(columns=['Product_id'])

In [4]:
df_numeric = df.apply(pd.to_numeric, errors='coerce')
df_tensor = torch.tensor(df_numeric.values, dtype=torch.float32)

In [111]:
X = torch.tensor(df.drop(['Price'], axis=1).values, dtype=torch.float32)
y = torch.tensor(df['Price'].values, dtype=torch.float32).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [112]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

/tmp/ipykernel_537/3241304788.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)
/tmp/ipykernel_537/3241304788.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.float32)


In [113]:
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x  

In [114]:
model = RegressionModel(X_train_scaled.shape[1])
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_scaled)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')
model.eval()
with torch.no_grad():
    y_pred = model(X_test_scaled)
    test_loss = criterion(y_pred, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [1/1000], Loss: 2207.6006
Epoch [2/1000], Loss: 2207.5742
Epoch [3/1000], Loss: 2207.5469
Epoch [4/1000], Loss: 2207.5186
Epoch [5/1000], Loss: 2207.4893
Epoch [6/1000], Loss: 2207.4597
Epoch [7/1000], Loss: 2207.4290
Epoch [8/1000], Loss: 2207.3979
Epoch [9/1000], Loss: 2207.3665
Epoch [10/1000], Loss: 2207.3337
Epoch [11/1000], Loss: 2207.3000
Epoch [12/1000], Loss: 2207.2654
Epoch [13/1000], Loss: 2207.2295
Epoch [14/1000], Loss: 2207.1924
Epoch [15/1000], Loss: 2207.1538
Epoch [16/1000], Loss: 2207.1140
Epoch [17/1000], Loss: 2207.0725
Epoch [18/1000], Loss: 2207.0300
Epoch [19/1000], Loss: 2206.9858
Epoch [20/1000], Loss: 2206.9402
Epoch [21/1000], Loss: 2206.8928
Epoch [22/1000], Loss: 2206.8433
Epoch [23/1000], Loss: 2206.7922
Epoch [24/1000], Loss: 2206.7393
Epoch [25/1000], Loss: 2206.6838
Epoch [26/1000], Loss: 2206.6262
Epoch [27/1000], Loss: 2206.5664
Epoch [28/1000], Loss: 2206.5039
Epoch [29/1000], Loss: 2206.4392
Epoch [30/1000], Loss: 2206.3718
Epoch [31/1000], Lo

# Tensorflow

In [24]:
import tensorflow as tf
import matplotlib as plt

In [25]:
from sklearn.model_selection import train_test_split
X = torch.tensor(df.drop(['Price'], axis=1).values, dtype=torch.float32)
y = torch.tensor(df['Price'].values, dtype=torch.float32).reshape(-1, 1)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = tf.constant(X_train_scaled, dtype=tf.float32)
X_test = tf.constant(X_test_scaled, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

learning_rate = 0.001 
num_features = X_train.shape[1]
w = tf.Variable(tf.random.normal([num_features, 1]))
b = tf.Variable(tf.zeros([1]))
def compute_loss():
    y_pred = tf.matmul(X_train, w) + b
    loss = tf.reduce_mean(tf.abs(y_train - y_pred))
    return loss

loss_values = []

for step in range(10000):
    with tf.GradientTape() as tape:
        loss = compute_loss()
    gradients = tape.gradient(loss, [w, b])
    w.assign_sub(learning_rate * gradients[0])
    b.assign_sub(learning_rate * gradients[1])

    loss_values.append(loss.numpy())
    
    if step % 100 == 0:
        print(f"Step: {step}, Loss: {loss.numpy()}")
def evaluate_model(X, y):
    y_pred = tf.matmul(X, w) + b
    loss = tf.reduce_mean(tf.abs(y - y_pred))
    return loss

test_loss = evaluate_model(X_test, y_test)
print(f"Test Loss: {test_loss.numpy()}")

Step: 0, Loss: 2207.7578125
Step: 100, Loss: 2207.65771484375
Step: 200, Loss: 2207.557861328125
Step: 300, Loss: 2207.457763671875
Step: 400, Loss: 2207.35791015625
Step: 500, Loss: 2207.2578125
Step: 600, Loss: 2207.15771484375
Step: 700, Loss: 2207.057861328125
Step: 800, Loss: 2206.957763671875
Step: 900, Loss: 2206.85791015625
Step: 1000, Loss: 2206.7578125
Step: 1100, Loss: 2206.65771484375
Step: 1200, Loss: 2206.557861328125
Step: 1300, Loss: 2206.457763671875
Step: 1400, Loss: 2206.35791015625
Step: 1500, Loss: 2206.2578125
Step: 1600, Loss: 2206.15771484375
Step: 1700, Loss: 2206.057861328125
Step: 1800, Loss: 2205.957763671875
Step: 1900, Loss: 2205.85791015625
Step: 2000, Loss: 2205.7578125
Step: 2100, Loss: 2205.65771484375
Step: 2200, Loss: 2205.557861328125
Step: 2300, Loss: 2205.457763671875
Step: 2400, Loss: 2205.35791015625
Step: 2500, Loss: 2205.2578125
Step: 2600, Loss: 2205.15771484375
Step: 2700, Loss: 2205.057861328125
Step: 2800, Loss: 2204.957763671875
Step: 290